<a href="https://colab.research.google.com/github/tftr0141/AIart/blob/main/voicerecognition/VCW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VCW: Voice-Cut with Whisper
openai-whisperをベースにしたボイスカットツールです。<br>
音声を文章毎に切り分ける作業を自動化できます。<br>

In [ ]:
#@title 環境の確認
!nvidia-smi

In [ ]:
#@title ライブラリのインストール
!pip install pydub
!pip install git+https://github.com/openai/whisper.git

import os
import glob
import whisper
from pydub import AudioSegment
import datetime
from tqdm import tqdm

In [ ]:
#@title Google Driveへのマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown ### 音声ファイルかフォルダを指定
#@markdown フォルダを指定した場合、複数の音声ファイルを一括で処理します。
audio_path = "/content/drive/MyDrive/"#@param {type:"string"}

#@markdown ### テキストファイルを出力するか
#@markdown チェックを入れた場合、音声を文章ごとに文字起こしして出力します。
output_txt = True #@param {type:"boolean"}

#@markdown ### ログを出力するか
#@markdown ログにはセリフの開始時間と終了時間が記載されます
output_log = True #@param {type:"boolean"}

#@markdown ### モデルサイズ
#@markdown 大きいほうが正確ですが、時間がかかります。
model_size = "medium" #@param ["tiny", "base", "small", "medium", "large"] {allow-input: false}

audio_path_list = []
if os.path.isdir(audio_path):
  audio_path_list = glob.glob(os.path.join(audio_path, "*"))
else:
  audio_path_list.append(audio_path)
audio_path_list.sort()
print("ボイスカットする音声ファイル一覧：")
for audio in audio_path_list:
  print(audio)

print("\nモデルの準備中…")
model = whisper.load_model(model_size)
print("完了！")

In [ ]:
#@title ボイスカット処理
dt_now = datetime.datetime.now()
output_dir_parent = dt_now.strftime('/content/drive/MyDrive/VCW-output/%Y-%m-%d-%H-%M-%S')
os.makedirs(output_dir_parent, exist_ok=True)

print("ファイルの出力先：" + output_dir_parent)

for i, path in enumerate(audio_path_list):
  output_dir_child = os.path.splitext(os.path.basename(path))[0]
  output_dir = os.path.join(output_dir_parent, output_dir_child)
  os.makedirs(output_dir)

  print("["+str(i+1)+"/"+str(len(audio_path_list))+"]")
  result = model.transcribe(audio=path, verbose=False, language='Japanese')

  audio = AudioSegment.from_file(path)
  log = ""
  print("文章数："+str(len(result['segments'])))
  for j, segment in tqdm(enumerate(result['segments'])):
    start = int(float(segment['start']))
    end = int(float(segment['end']))
    text = segment['text']

    start_m, start_s = divmod(start, 60)
    start_h, start_m = divmod(start_m, 60)
    start_h_m_s = f'start: {str(start_h).zfill(2)}:{str(start_m).zfill(2)}:{str(start_s).zfill(2)}'
    end_m, end_s = divmod(end, 60)
    end_h, end_m = divmod(end_m, 60)
    end_h_m_s = f'end: {str(end_h).zfill(2)}:{str(end_m).zfill(2)}:{str(end_s).zfill(2)}'

    log += '[' + str(j).zfill(5) + ' ' + start_h_m_s + ' ' + end_h_m_s + f'] {text}\n'

    cropped_audio = audio[start*1000:end*1000]
    cropped_audio.export(os.path.join(output_dir, str(j).zfill(5) + '.mp3'))

    if output_txt:
      with open(os.path.join(output_dir, str(j).zfill(5) + '.txt'), 'w') as f:
        f.write(segment['text'])
  
  if output_log:
    log_path = os.path.join(output_dir_parent, 'log_' + output_dir_child + '.txt')
    with open(log_path, 'w') as f:
      f.write(log)

print('\nEnjoy.')